In [1]:
import argparse
import sys
import functions as func
import operator
import pandas as pd
import numpy as np
import re
import Reader

In [16]:
def parseData(args):
    polarity_contents = func.parsePolarityOut(args.polarity_simpl_file, term2POS)    #'data/polarity_simpl.out', term2POS)
    aspect_contents = func.parseAspectOut(args.aspect_simpl_file, term2POS)     #'data/aspect_simpl.out', term2POS)
    testReview_contents = func.parseTestReviewOut(args.test_simpl_file, term2POS)#'data/test_simpl.out', term2POS)

    contents = polarity_contents+aspect_contents+testReview_contents
    return contents

# Train Word2Vec model
def trainWord2Vec(contents):
    model = func.train_word2vec_model(contents)
    return model

# Find similar terms
def findSimilarTermsOfAllAspects(model, serviceTerms, envTerms, priceTerms, trafficTerms, restaurantTerms, threshold):
    serviceNewTerms = func.getSimTerms(model, serviceTerms, [threshold, threshold, threshold])
    envNewTerms = func.getSimTerms(model, envTerms, [threshold, threshold, threshold, threshold])
    priceNewTerms = func.getSimTerms(model, priceTerms, [threshold, threshold])
    trafficNewTerms = func.getSimTerms(model, trafficTerms, [threshold, threshold])
    restaurantNewTerms = func.getSimTerms(model, restaurantTerms, [threshold, threshold])
    return serviceNewTerms, envNewTerms, priceNewTerms, trafficNewTerms, restaurantNewTerms


# DecideAspect for each sentence and write to file
def decideAspectToFile(arg, serviceNewTerms, envNewTerms, priceNewTerms, trafficNewTerms, restaurantNewTerms, filename):
    #f = open('Aspect_per_sentence.csv', 'w')
    f = open(filename, 'w')
    idsentences = Reader.Reader.test(args.test_simpl_file)
    for Id, sentences in idsentences:
        for i,sentence in enumerate(sentences):
            f.write(str(Id)+","+str(func.decideAspect(sentence, serviceNewTerms, envNewTerms, priceNewTerms, trafficNewTerms, restaurantNewTerms))+'\n')
    f.close()

    '''
    for Id, comment in testReviewDict.items():
        sentences = func.split2sentences(comment)
        print(sentences)
        for i,sentence in enumerate(sentences):
            print(i,sentence)
            f.write(str(Id)+","+str(func.decideAspect(sentence, serviceNewTerms, envNewTerms, priceNewTerms, trafficNewTerms, restaurantNewTerms))+'\n')
    f.close()
    '''



# Count label (0 or not 0) according to input file
def decideMentionTofile(testReviewDict, testDF, serviceNewTerms, envNewTerms, priceNewTerms, trafficNewTerms, restaurantNewTerms):
    f = open('result.csv', 'w')
    f.write('Id,Label\n')
    zeroCnt = 0
    for i in range(testDF.shape[0]):
        Id, Review_id, Aspect = testDF.iloc[i]
        ans = func.decideMention(testReviewDict[int(Review_id)], str(Aspect), serviceNewTerms, envNewTerms, priceNewTerms, trafficNewTerms, restaurantNewTerms)
        if ans == 0:
            zeroCnt += 1
        f.write(str(Id)+","+str(ans)+'\n')
    print('zeroCnt', float(zeroCnt/testDF.shape[0]))
    f.close()

    
class Args:
    def __init__(self, polarity_simpl_file, aspect_simpl_file, test_simpl_file, question_file, output_file, threshold):
        self.polarity_simpl_file = polarity_simpl_file
        self.aspect_simpl_file = aspect_simpl_file
        self.test_simpl_file = test_simpl_file
        self.question_file = question_file
        self.output_file = output_file
        self.threshold = threshold

In [3]:
args = Args('data/polarity_simpl.out', 'data/aspect_simpl.out', 'data/test_simpl.out', 'data/test.csv', 'Aspect_per_sentence.csv', 0.45)

In [4]:
# Term to POS tag
term2POS = func.readDictBig('external/dict.txt.big')
contents = parseData(args)
model = trainWord2Vec(contents)

In [5]:
# Head aspect terms
serviceTerms = ['服务','态度','人员']
envTerms = ['环境','客房','设备','空调']
priceTerms = ['价格','房价']
trafficTerms = ['交通','地理']
restaurantTerms = ['餐厅','早餐']


# Read input file for submission
testReviewDict = func.readTestReview(args.test_simpl_file)    #'data/test_simpl.out')
testDF = pd.read_csv(args.question_file)                     #'data/test.csv')

In [9]:
#serviceNewTerms, envNewTerms, priceNewTerms, trafficNewTerms, restaurantNewTerms = findSimilarTermsOfAllAspects(model, serviceTerms, envTerms, priceTerms, trafficTerms, restaurantTerms, float(args.threshold))
serviceNewTerms, envNewTerms, priceNewTerms, trafficNewTerms, restaurantNewTerms = findSimilarTermsOfAllAspects(model, serviceTerms, envTerms, priceTerms, trafficTerms, restaurantTerms, 0.4)

print(len(serviceNewTerms))
print(len(envNewTerms))
print(len(priceNewTerms))
print(len(trafficNewTerms))
print(len(restaurantNewTerms))

167
381
126
66
239


In [14]:
#print(serviceNewTerms)
#print(envNewTerms)
#print(priceNewTerms)
#print(trafficNewTerms)
#print(restaurantNewTerms)
#func.findSimTerm(model, '服务')

In [17]:
decideAspectToFile(args, serviceNewTerms, envNewTerms, priceNewTerms, trafficNewTerms, restaurantNewTerms, args.output_file)
#decideMentionTofile(testReviewDict, testDF, serviceNewTerms, envNewTerms, priceNewTerms, trafficNewTerms, restaurantNewTerms)

print('End')

End
